# Assignment - Visualization

Make sure to **read the instructions** completely. Failing to follow the instructions will have a negative impact on your grade!

In this assignment, we will explore ways to visualize the data, starting with simple plots for early exploration and build towards more interactive methods. As we want to go beyond static plots possible with for example matplotlib or seaborn, we will use the [Bokeh](https://www.bokeh.org/) library. This means the exercises **must** be implemented with Bokeh to receive points.

This assignment includes four parts (Assignmets V.1 to V.4) which provide different amounts of points. V.1 to V.3 are independent assignments, but V.4 builds on the previous assignments.
 
**Before you submit**, make sure that you are not accidentaly using any global variables. Restart the kernel (wiping all global variables) and run the code from top to bottom by clicking "Kernel" => "Restart & Run all" in the menu bar at the top.

### API documentation

note: you can find the used versions by checking the `environment.yml` file or `running conda list -n dsait4000vis` from the command line

* [Bokeh](https://docs.bokeh.org/) (also has examples)
* [Pandas](https://pandas.pydata.org/docs/)
* [NumPy](https://numpy.org/doc/stable/)
* [SciPy](https://docs.scipy.org/doc/scipy/)


### Rules
For this assignment the following rules apply:

**General**
 * The assignment should be completed in the **groups** you have already assembled for the previous assignment. Work together on all parts and do not distribute the work. Everyone is responsible for everything. That also means if there is fraud in a part it will be considered fraud by the whole group and voids the complete assignment, even if you distributed the parts of the assignment.
 * Any kind of intergroup discussion will be considered fraud and both the parties will be punished.
 * All code must be written within group. All external help, with the exception of Python/library documentation and the lecture slides, will be considered fraud.
 * Do not add any libarires to those already included notebook (i.e., `numpy`, `scipy`, `pandas`, and `bokeh`)
 * Add functionality from bokeh, like ```from bokeh.io import output_notebook```, on top of the block where you need it.
 * **Do not rename the notebook file**

**Grading**
 * Exercise may (partially) be graded automatically, using additional hidden tests. These tests will check your solution for different (unseen) inputs.
 * If these tests pass without error (warnings are allowed) then you receive full points.
 * Do not make any assumptions on the input data (such as number or type of columns) unless specified otherwise. Doing so may result in the tests failing and thus 0 points.
 * Your grade is calculated as the sum of the achieved points and will be rounded to the closest 0.1 point.
 
**Submissions**
 * Submit your code to Brightspace
 * For this assignment you must *only* submit the notebook (`*.ipynb`) file filled in with your solution.
 * Refer to Brightspace for the exact deadline.
 * We will consider submissions that are handed in up to 3 hourse after the deadline but with a penalty as follows:
 * For every completed 20 minutes past the deadline the grade will be reduced by 0.5 (e.g., with the deadline at 17:00, a submission at 17:19 will get the full grade, a submission at 17:20 will receive a 0.5 grade penalty.

In [1]:
# We do the basic setup here.
# Do not (!) change this cell! All changes will be reset during grading!

# numpy and pandas
import numpy as np
import pandas as pd

# We import a range of bokeh functionality that will likely be needed already
# if you need more, import it at the top of the corresponding cell
from bokeh.plotting import figure, gridplot, show
from bokeh.layouts import column
from bokeh.transform import factor_cmap, linear_cmap
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter, BasicTicker, PrintfTickFormatter, CustomJS, Select, axes
from bokeh.models.annotations import Label
from bokeh.io import output_notebook
output_notebook() # activate Bokeh output to Jupyter notebook

# You can use this to calculate the density plot in Assignment V.3
from scipy.stats import gaussian_kde

# We load the adult data  (https://archive.ics.uci.edu/dataset/2/adult) as in previous assignments (note, we keep the headers, to use them in the pandas dataframe)
# Note: we do not change ? values to nan, but rather keep them as ? so they will be shown in the visualizations
data = pd.read_csv("datasets/Visualization/adult_all.csv")

data.head()

Loading BokehJS ...

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Assignment V.1 (5 points)
Getting an overview of the data distribution

In this first assignment, we want to use visualization to get a quick overview of the distribution of the different attributes/columns. This can often be helpful to make sure that we use models that match the distribution downstream or to quickly identify for example whether the data is distributed over all classes equally if the attribute is intended to be used for training a model.

A good way to quickly plot the distribution is a simple bar chart for the column. However, this will only allow us to look at one attribute at a time. In class we have discussed small multiples. We can use this concept to plot all attributes in a grid of plot. 

For this assignment do the following:
* create a grid of bar charts, with one chart per column in the dataset. The attribute values shall be on the x-axis and the count of each value on the y-axis. Look at pandas `value_counts` function to get the latter.
  Hint: The `figure()` function works slightly different for numerical and categirucal data.
* make sure that the charts and axes are well labeled (for example like in the screenshot below). It will be hard to properly label the x-axis with many items (e.g., native-country). Do this as good as you can, you will likely not find a perfect solution for all attributes.
* add a tooltip to show the values on hover (see the education chart, top-right, in the image below)

The result should look similar to the below.

[<img src="assets/barcharts.png" alt="Small Multiples showing the distributions of all columns in the dataset." width="500"/>](assets/barcharts.png)

In [3]:
# YOUR CODE HERE
def plot_dataframe( df, grid_length=4, subfig_height=100, fig_width=400, margin=0.02, bar_width=0.8 ):
    figs = []
    grid_row = [ None for _ in range(grid_length) ]
    idx = 0
    for column in df.columns:
        if idx == grid_length:
            idx = 0
            figs.append(grid_row)
            grid_row = [ None for _ in range(grid_length) ]

        is_categorical = df.dtypes[column]==object
        counts = df[column].value_counts( sort=is_categorical )

        x = counts.index.values
        y = counts.values
        margin_y = 0 - margin * max(y)

        fig = figure( height=subfig_height, title=column.capitalize(), toolbar_location=None, tools="hover", tooltips=[(column.capitalize(),"@x"), ("Value","@top")] )
        if is_categorical:
            fig = figure( x_range=x, height=subfig_height, title=column.capitalize(), toolbar_location=None, tools="hover", tooltips=[(column.capitalize(),"@x"), ("Value","@top")] )
        fig.vbar( x=x, top=y, width=bar_width, bottom=0 )
        
        fig.xaxis.major_label_orientation = 0.8
        fig.y_range.start = margin_y
        grid_row[idx] = fig
        idx += 1
    
    figs.append(grid_row)
    
    grid = gridplot( figs, width=fig_width, height=len(figs)*subfig_height)
    show(grid)
    
plot_dataframe(data)

### Assignment V.2 (5 points)

Getting an overview of possible correlations and some interactivity.

To keep this simply we only use the numerical variables here. You can use pandas `corr()` [function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html) directly on the dataframe using the `numeric_only = True` argument. The output will contain only the results from using numerical columns. It should be noted though, that there are also ways to calculate dependencies or correlation between catagorical (e.g., [Chi-Square_test](https://en.wikipedia.org/wiki/Chi-square_test), [Cramér's V](https://en.wikipedia.org/wiki/Cramér%27s_V)) or categorical and numerical (e.g., [One-way ANOVA test](https://en.wikipedia.org/wiki/F-test#One-way_ANOVA_example)) values.

The output will be a matrix if pairwise correlations. It is common, to visualize such data in a heatmap (images below).

* Calculate the correlation of the numerical columns using pandas `corr()`, using Pearson correlation (this is the default of the function).
* Create a heatmap showing the values. The data format output by `corr()` might not be very useful. Look for example at [pandas `stack()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html) for possible ways to transforma the data into a typical dataframe.
* A heatmap uses x and y position for the attributes and color for the value, choose a fitting colormap and **justify it below in 50 words or less**. (Note: the color chosen in the example below might or might not be good. Do **not** use the example to justify your choice.)
* Make sure the chart is legible by adding axes, legends, a color bar, or similar. It is up to you to decide what is needed.

We have now visualized one possible correlation. We want to extend our chart to be able to switch between the Pearson and Spearman correlation interactively using a dropdown menu directly in the chart.
* Add a `Select` dropdown with the two options.
* Calculate both correlation values and add a plot for each to your figure.
* Use `js_on_change` on the added Select to add code to switch between the two plot

[<img src="assets/heatmap.png" alt="Heatmap showing correlations" width="500"/>](assets/heatmap.png)
[<img src="assets/heatmap_select.png" alt="Heatmap showing correlations with open selection widget" width="500"/>](assets/heatmap_select.png)

In [4]:
# We calculate both matrices
correlation_matrices = {
    "pearson": data.corr(numeric_only = True, method="pearson"),
    "spearman": data.corr(numeric_only = True, method="spearman")
}

# this is the default correlation method
correlation_method = "spearman"

# YOUR CODE HERE

In [5]:

def cramers_V( X ):
    from scipy.stats import chi2_contingency
    """Compute Cramér's V between categorical columns."""
    cols = []
    for column in X.columns:
        if X[column].dtypes==object:
            cols.append(column)
    
    cramer_matrix = pd.DataFrame(index=cols, columns=cols, dtype=float)
    for col1 in cols:
        for col2 in cols:
            conf_mat = pd.crosstab(X[col1], X[col2])
            chi_sq = chi2_contingency(conf_mat)[0]
            n = conf_mat.sum().sum()
            V = chi_sq / (n * (min(conf_mat.shape) - 1))
            cramer_matrix.loc[col1, col2] = np.sqrt(V)
    return cramer_matrix


In [6]:

def create_color_map():
    from bokeh.colors import RGB
    
    neg_color = RGB( 0, 0, 255 )
    pos_color = RGB( 255, 0, 0 )
    
    colors_low = []
    colors_high = []
    for i in range(1,129):
        colors_high.append( RGB( max( pos_color.r, 255 - 2*i ), max( pos_color.g, 255 - 2*i ), max( pos_color.b, 255 - 2*i ) ) )
        colors_low.append( RGB( max( neg_color.r, 255 - 2*i ), max( neg_color.g, 255 - 2*i ), max( neg_color.b, 255 - 2*i ) ) )

    return list(reversed(colors_low)) + colors_high


In [23]:

def make_heatmap( matrices, default_corr ):
    assert default_corr in matrices
    
    sources = {}
    cols = {}
    for k in matrices:
        matrix = matrices[k]

        cols[k] = {
            "x_cols": list( matrix.columns ),
            "y_cols": list( reversed(matrix.columns) )
        }
        
        matrix = pd.DataFrame( matrix.stack(), columns=["corr"] ).reset_index()
        data = {
            "x": matrix['level_0'].values,
            "y": matrix['level_1'].values,
            "corr": matrix['corr'].values
        }
        sources[k] = ColumnDataSource(data)

    fig = figure( title=f"Correlation",
                  x_range=cols[default_corr]['x_cols'], y_range=cols[default_corr]['y_cols'],
                  width=700, height=700,
                  tools="hover,save,pan,box_zoom,reset,wheel_zoom", toolbar_location='above',
                  tooltips=[('Column:', "@x"),('Row:', "@y"),('Value:', "@corr")],
                  x_axis_location='above'
                )
    
    fig.grid.grid_line_color = None
    fig.axis.axis_line_color = None
    fig.axis.major_tick_line_color = None
    fig.axis.major_label_standoff = 0
    fig.xaxis.major_label_orientation = 0.8
    fig.yaxis.major_label_orientation = 0.8

    colors = create_color_map()    
    mapper = linear_cmap( "corr", colors, low=-1, high=1)
    heatmap = fig.rect( x="x", y="y", width=1, height=1, source=sources[default_corr], fill_color=mapper, line_color=None )
    
    selector = Select( title="Correlation:", value=default_corr, options=[ k for k in sources ] )
    
    callback = CustomJS( args=dict( sources=sources, fig=fig, cols=cols, heatmap=heatmap ), code="""
        const s_key = cb_obj.value;
        const s_data = sources[s_key].data
        fig.x_range.factors = cols[s_key].x_cols;
        fig.x_range.change.emit();
        fig.y_range.factors = cols[s_key].y_cols;
        fig.y_range.change.emit();
        heatmap.data_source.data = s_data;
        heatmap.data_source.change.emit();
    """ )

    selector.js_on_change("value", callback)
    
    fig.add_layout( heatmap.construct_color_bar(
                        ticker=BasicTicker(),
                        border_line_color=None,
                    ), 'right'
                  )
    
    show(column(selector,fig))


In [24]:
correlation_matrices["cramers_v"] = cramers_V(data)

make_heatmap( correlation_matrices, correlation_method )

#### Response: Justification for colormap
**_Add your justification for the choice of colormap below._**

Correlation is a measure of how "related" two features are. A high absolute value shows a high amount of statistical dependence while a low one shows independence. We show both kinds of dependence with different colors to make the distinction as clear as possible.

### Assignment V.3 (5 points)

Correlation in detail.

Pick the any two numerical and one categorical column from the dataset. We already extract the names of the first two numerical and first categorical columns below, the easiest is to just use those. You can make these also user-selectable like in the previous assignment, but you do not have to for this assignment. (Remenber that we might test this with a different dataset, so do not pick by name.)

The goal here is to visualize the complete data of the selected columns to get more detail on the correlation. We will proceed in two steps again.

In the first step, create a scatterplot to visualize the selected columns.
* A scatterplot is a common representation, bokeh provides it using `scatter`.
* Use the x and y axes for the numerical values.
* Choose a proper encoding for the categorical column and  **justify** it in the field below the code **in 50 words or less**. (Note: this encoding was omitted in the example figure below.)
* Make sure the chart is legible by adjusting mark sizes, adding axes, legends, a color bar, or similar, etc. It is up to you to decide what is needed.

Even with putting in a lot of effort, you will notice that there will be a lot of overplotting, even with 'just' ~50k points in the dataset. In the example below, you see iso-density lines added to show how the points distribute. We have already added the `kde` function to [estimate the density.](https://en.wikipedia.org/wiki/Multivariate_kernel_density_estimation) Use this to calculate the point density. This is taken from an [example in the Bokeh documenation](https://docs.bokeh.org/en/latest/docs/examples/topics/stats/kde2d.html). Look at the [example](https://docs.bokeh.org/en/latest/docs/examples/topics/stats/kde2d.html) and adjust it to your needs.
* Add the density information to your plot.

[<img src="assets/scatterplot.png" alt="Scatterplot with density indicator" width="700"/>](assets/scatterplot.png)

In [ ]:
# kde calculation taken from https://docs.bokeh.org/en/latest/docs/examples/topics/stats/kde2d.html
# look at the example on how it is used. You can use the plot code also for inspiration, but make sure to adjust it for proper results
# note, N is the kernel size, larger values will make this lengthy to calculate but give more smooth results.
# When you use it consider using small values for N during testing.
def kde(x, y, N):

    xmin, xmax = x.min(), x.max()
    ymin, ymax = y.min(), y.max()

    X, Y = np.mgrid[xmin:xmax:N*1j, ymin:ymax:N*1j]
    positions = np.vstack([X.ravel(), Y.ravel()])
    values = np.vstack([x, y])
    kernel = gaussian_kde(values)
    Z = np.reshape(kernel(positions).T, X.shape)

    return X, Y, Z

# we extract the names of the first two numerical columns
num_cols = []
num_cols.append(data.select_dtypes(include="number").columns[0])
num_cols.append(data.select_dtypes(include="number").columns[1])
# and the first categorical column.
cat_col = data.select_dtypes(include=["object", "category"]).columns[0]
# You can just use the above columns to set up your visualization. Feel free to change these for testing.
# For the submission, you can safely assume that at least 2 numerical and 1 categorical columns are available in the dataset.

# YOUR CODE HERE
raise NotImplementedError()

#### Response: Justification for categorical data encoding
**_Add your justification for the choice of encoding here!_**

YOUR ANSWER HERE

### Assignment V.4 (10 points)

Interactive exploration

In this final assignment we want to do some true interactive exploration harnessing the power of Bokeh. This might allow us to identify patterns related to specific subsets of the data.
You have two options to proceed, based on your previous results. The below descriptions are minimal solutions, feel free to go beyond the suggested functionality, if you deem it useful=. If you do so add a brief list of what you added in the textfield below.

#### Pick one of the the following options:

1. Use the result of **Assignment V.1** and extend it such that you can explore subsets of the data based on selection in one of the histograms. I.e.,
   * Make it possible to select any subset of the data by clicking the corresponding bar in any of the bar chart. E.g., clicking on the *Bachelors* bar in the *Education* chart should select all datapoints, where the value of Education is Bachelors.
   * highlight the selection in all charts (while keeping the full dataset). E.g., render a second bar corresponding to the fraction of the corresponding points with in a different color on top of the original bar.
<br/><br/>
2. Combine the results of **Assignment V.2** and **Assignment V.3** to an interactive overview+detail plot and allow filtering the scatterplot to show only selected categories of the categorical column.
   * Show both plots next to each other in a grid.
   * Allow seleciton of the numerical columns in the scatterplot by clicking on a cell in the heatmap.
   * Allow selection of the categorical column (e.g., by a dropdown, as in Assignment V.2).
   * Add a way to select one or multiple values of the categorical coloumn and filter the scatterplot based on those. E.g., Plot only the datapoints where *Education* equals *Bachelors*, when Education is selected as the categorical variable of the plot.

#### Some possibly useful hints:
* Check out **TapTool** for more information on enabling clicking events on plots
* Have a look at **CDSView** for one possible way to filter out a subset of data on a plot
* [Enabling click callbacks on bars](https://docs.bokeh.org/en/2.4.3/docs/user_guide/tools.html#setting-the-active-tools)
* [Getting clicked bar index in callback functions](https://discourse.bokeh.org/t/a-different-plot-in-another-figure-when-a-bar-of-a-barchart-is-clicked/8572/2)
* m[Apply hover tools to plots](https://stackoverflow.com/questions/29435200/bokeh-plotting-enable-tooltips-for-only-some-glyphs)

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE